In [4]:

import pandas as pd
messages = pd.read_csv('/Users/pranjalparmar/Documents/git-repos/NLP/spam.csv', encoding='latin-1')
messages.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True, errors='ignore')
messages.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)


In [5]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
## Data Cleaning and Preprocessing

import re
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranjalparmar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords


In [8]:
print(f"Total messages loaded: {len(messages)}")


Total messages loaded: 5572


In [9]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

def fix_spaced_words(text):
    # Join sequences like "g o   u n t i l" into "go until"
    return re.sub(r'((?:\b[a-zA-Z]\b\s*){2,})',
                  lambda m: m.group(0).replace(' ', ''), text)

def pos_lemmatize(tokens, lemmatizer):
    pos_tags = nltk.pos_tag(tokens)  # tag the list of tokens directly
    lemmatized_words = []
    for word, tag in pos_tags:
        
        if tag.startswith('NN'):
            lemmatized_word = lemmatizer.lemmatize(word, 'n')
        elif tag.startswith('VB'):
            lemmatized_word = lemmatizer.lemmatize(word, 'v')
        elif tag.startswith('JJ'):
            lemmatized_word = lemmatizer.lemmatize(word, 'a')
        elif tag.startswith('RB'):
            lemmatized_word = lemmatizer.lemmatize(word, 'r')
        else:
            lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_words.append(lemmatized_word)
    return lemmatized_words


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pranjalparmar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pranjalparmar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pranjalparmar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
empty_count = 0
for i in range(len(messages)):
    review = messages['message'][i]
    review = fix_spaced_words(review)
    review = re.sub(r'[^\x00-\x7F]+', ' ', review)
    review = re.sub('[^a-zA-Z\s]', ' ', review).lower()
    tokens = review.split()
    lemmatized_words = pos_lemmatize(tokens, lemmatizer)
    filtered_words = [w for w in lemmatized_words if w not in set(stopwords.words('english'))]
    if len(filtered_words) == 0:
        empty_count += 1
print(f"Number of messages that became empty or lost all meaningful words: {empty_count}")


<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
/var/folders/_7/wbb8p5rn1jxcc1d45rrn7qqh0000gn/T/ipykernel_15958/425627394.py:6: SyntaxWarning: invalid escape sequence '\s'
  review = re.sub('[^a-zA-Z\s]', ' ', review).lower()


Number of messages that became empty or lost all meaningful words: 10


In [11]:

import re
corpus = []
for i in range(len(messages)):
    review = messages['message'][i]
    review = fix_spaced_words(review)  
    review = re.sub(r'[^\x00-\x7F]+', ' ', review)
    review = re.sub('[^a-zA-Z\s]', ' ', review).lower()
    tokens = review.split()
    lemmatized_words = pos_lemmatize(tokens, lemmatizer)
    filtered_words = [word for word in lemmatized_words if word not in set(stopwords.words('english'))]

    cleaned_review = ' '.join(filtered_words)
    corpus.append(cleaned_review)
print(f"Total messages after preprocessing: {len(corpus)}")


<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/var/folders/_7/wbb8p5rn1jxcc1d45rrn7qqh0000gn/T/ipykernel_15958/2988841882.py:7: SyntaxWarning: invalid escape sequence '\s'
  review = re.sub('[^a-zA-Z\s]', ' ', review).lower()


Total messages after preprocessing: 5572


In [12]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine get amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor ucalready say',
 'nah think go usf live around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner value network customer select receivea prize reward claim call claim code kl valid hour',
 'mobile month urentitled update late colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cry enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent win week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'sea

In [13]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, binary=True)

In [14]:
X = cv.fit_transform(corpus).toarray()
X.shape
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5572, 2500))

In [15]:
cv.vocabulary_


{'go': np.int64(858),
 'point': np.int64(1620),
 'crazy': np.int64(499),
 'available': np.int64(137),
 'bugis': np.int64(274),
 'great': np.int64(880),
 'world': np.int64(2444),
 'la': np.int64(1144),
 'buffet': np.int64(273),
 'cine': np.int64(394),
 'get': np.int64(846),
 'wat': np.int64(2377),
 'ok': np.int64(1514),
 'lar': np.int64(1155),
 'wif': np.int64(2411),
 'oni': np.int64(1522),
 'free': np.int64(795),
 'entry': np.int64(666),
 'wkly': np.int64(2432),
 'comp': np.int64(440),
 'win': np.int64(2415),
 'cup': np.int64(517),
 'final': np.int64(751),
 'tkts': np.int64(2190),
 'st': np.int64(2032),
 'may': np.int64(1327),
 'text': np.int64(2145),
 'receive': np.int64(1724),
 'question': np.int64(1688),
 'std': np.int64(2043),
 'txt': np.int64(2245),
 'rate': np.int64(1705),
 'apply': np.int64(98),
 'dun': np.int64(631),
 'say': np.int64(1820),
 'early': np.int64(636),
 'nah': np.int64(1449),
 'think': np.int64(2162),
 'usf': np.int64(2312),
 'live': np.int64(1219),
 'around': np.i

## N-Gram Implementation


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, binary=True, ngram_range=(2,3))
X = cv.fit_transform(corpus).toarray()
cv.vocabulary_

{'ok lar': np.int64(1462),
 'free entry': np.int64(629),
 'entry wkly': np.int64(553),
 'wkly comp': np.int64(2438),
 'cup final': np.int64(415),
 'std txt': np.int64(1952),
 'txt rate': np.int64(2173),
 'rate apply': np.int64(1678),
 'free entry wkly': np.int64(631),
 'std txt rate': np.int64(1953),
 'txt rate apply': np.int64(2174),
 'think go': np.int64(2074),
 'freemsg hey': np.int64(658),
 'like fun': np.int64(1090),
 'per request': np.int64(1528),
 'oru minnaminunginte': np.int64(1495),
 'nurungu vettam': np.int64(1447),
 'set callertune': np.int64(1829),
 'callertune caller': np.int64(203),
 'caller press': np.int64(199),
 'press copy': np.int64(1633),
 'copy friend': np.int64(393),
 'friend callertune': np.int64(662),
 'oru minnaminunginte nurungu': np.int64(1496),
 'nurungu vettam set': np.int64(1448),
 'set callertune caller': np.int64(1830),
 'callertune caller press': np.int64(204),
 'caller press copy': np.int64(200),
 'press copy friend': np.int64(1634),
 'copy friend cal

## TF-IDF 


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=300)

In [19]:
X = tfidf.fit_transform(corpus).toarray()

In [21]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))

In [22]:
X

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0.416, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.499, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.267, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.572, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 

In [23]:
tfidf = TfidfVectorizer(max_features=300,ngram_range=(2,3))
X = tfidf.fit_transform(corpus).toarray()

In [24]:
tfidf.vocabulary_

{'free entry': np.int64(83),
 'rate apply': np.int64(205),
 'think go': np.int64(249),
 'claim call': np.int64(42),
 'call claim': np.int64(14),
 'claim code': np.int64(43),
 'late colour': np.int64(140),
 'free call': np.int64(81),
 'call mobile': np.int64(25),
 'chance win': np.int64(41),
 'win cash': np.int64(290),
 'txt word': np.int64(263),
 'let know': np.int64(142),
 'feel like': np.int64(78),
 'reply yes': np.int64(212),
 'go home': np.int64(99),
 'anything lor': np.int64(3),
 'call reply': np.int64(31),
 'mobile free': np.int64(163),
 'free camcorder': np.int64(82),
 'please call': np.int64(189),
 'lt gt': np.int64(153),
 'miss call': np.int64(160),
 'want go': np.int64(284),
 'first time': np.int64(80),
 'like lt': np.int64(143),
 'like lt gt': np.int64(144),
 'sorry call': np.int64(227),
 'call later': np.int64(23),
 'sorry call later': np.int64(228),
 'award bonus': np.int64(8),
 'prize call': np.int64(200),
 'send ur': np.int64(221),
 'ur award': np.int64(268),
 'call free